In [2]:
# Import necessary libraries

import pandas as pd # For data manipulation and analysis
import numpy as np # For numerical operations
import pickle # For serializing and deserializing Python objects
from modelling_utils import train_knn_classifier, evaluate_model, plot_model_comparison, split_shuffle_data

In [3]:
# Load features from a pickle file
feature_dict_path = "/home/suraj/Repositories/FM-extractors-radiomics/evaluation/features/dlcs.pkl" # Path to the pickle file containing features
with open(feature_dict_path, 'rb') as file: # Open the file in read binary mode
    data = pickle.load(file) # Load the data from the pickle file

In [6]:
# Store test accuracies for each model
test_accuracies_dict = {} # Initialize an empty dictionary to store test accuracies

# Iterate through each model's features
for model_name, values in data.items(): # Loop through each model and its corresponding features
    # Skip MedImageInsightExtractor
    if model_name == "MedImageInsightExtractor": # Skip the MedImageInsightExtractor model
        continue

    # Extract paths and labels for train, val, and test sets
    train_labels = [v["row"]["Malignant_lbl"] for v in values["train"]] # Extract malignancy labels for the training set
    val_labels = [v["row"]["Malignant_lbl"] for v in values["val"]] # Extract malignancy labels for the validation set
    
    # Stack features
    train_items = np.vstack([v["feature"] for v in values["train"]]) # Stack features for the training set
    val_items = np.vstack([v["feature"] for v in values["val"]]) # Stack features for the validation set
    
    # Concatenate train and val
    all_items = np.vstack((train_items, val_items))
    all_labels = train_labels + val_labels

    # Average across multiple shuffle splits
    n_splits = 5
    split_scores = []
    
    for split in range(n_splits):
        # Get stratified indices for new train/val split with different random seeds
        train_items, train_labels, val_items, val_labels, test_items, test_labels = split_shuffle_data(
            all_items, all_labels, train_ratio=0.5, val_ratio=0.1, random_seed=10+split
        )

        # Train model with hyperparameter optimization
        best_model, study = train_knn_classifier(train_items, train_labels, val_items, val_labels)
        
        # Get score for this split
        split_score = evaluate_model(best_model, test_items, test_labels)
        split_scores.append(split_score)
    
    # Average the scores across splits
    avg_score = np.mean(split_scores)
    
    # Evaluate on test set
    test_accuracies_dict[model_name] = avg_score

[I 2025-01-23 16:57:39,543] A new study created in memory with name: no-name-e5cc45d2-05a6-4a30-8342-3bc8a0ca20f3
[I 2025-01-23 16:57:39,740] Trial 0 finished with value: 0.6864612188365651 and parameters: {'k': 29}. Best is trial 0 with value: 0.6864612188365651.
[I 2025-01-23 16:57:39,953] Trial 1 finished with value: 0.6018005540166206 and parameters: {'k': 12}. Best is trial 0 with value: 0.6864612188365651.
[I 2025-01-23 16:57:40,146] Trial 2 finished with value: 0.6016274238227146 and parameters: {'k': 11}. Best is trial 0 with value: 0.6864612188365651.
[I 2025-01-23 16:57:40,266] Trial 3 finished with value: 0.6681094182825484 and parameters: {'k': 42}. Best is trial 0 with value: 0.6864612188365651.
[I 2025-01-23 16:57:40,368] Trial 4 finished with value: 0.5832756232686981 and parameters: {'k': 3}. Best is trial 0 with value: 0.6864612188365651.
[I 2025-01-23 16:57:40,437] Trial 5 finished with value: 0.6923476454293628 and parameters: {'k': 28}. Best is trial 5 with value: 0

In [7]:
# Plot test accuracies
fig = plot_model_comparison(test_accuracies_dict)
fig.show() # Show the plot